In [5]:
from util import load_eidc_data as eidc

df = eidc.load_as_df('data/catalogue_metadata.json')
df

,title,description,lineage
0,"Land Cover Map 2007 (25m raster, NI)",LCM2007 is a parcel-based thematic classificat...,LCM2007 uses a spatial framework based on OSNI...
1,Land Cover Map 2021 (25m rasterised land parce...,This is a 25m pixel data set representing the ...,UKCEH’s automated land cover algorithms classi...
2,"Land Cover Map 2000 (vector, GB)",This dataset consists of the vector version of...,"In a major development of earlier methods, LCM..."
3,"Land Cover Map 2021 (10m classified pixels, N....",This is a 10m pixel data set representing the ...,UKCEH’s automated land cover algorithms classi...
4,Telomere length data from a wild population of...,These data consist of relative telomere length...,Blood samples (~25 μl) were collected and stor...
...,...,...,...
1862,Potential evapotranspiration derived from HadU...,Gridded potential evapotranspiration calculate...,This dataset was derived from the HadUK-Grid g...
1863,Fine root production in human-modified forests...,This dataset includes measurements of soil res...,We assessed roots production in 20 study plots...
1864,Nitrous oxide fluxes and associated soil measu...,This data set includes over 500 individual flu...,Flux measurements were made using a high-preci...
1865,A land use map of Peninsular Malaysia for the ...,Gridded land use map of Peninsular Malaysia wi...,The land use is based on detailed polygon land...


In [6]:
from transformers import pipeline

model_names = ['deepset/roberta-base-squad2', 
               'distilbert/distilbert-base-cased-distilled-squad', 
               'deepset/minilm-uncased-squad2',
               'deepset/bert-base-cased-squad2',
               'Intel/dynamic_tinybert']
qa_pipes = {}
for model_name in model_names:
    qa_pipes[model_name] = pipeline('question-answering', model=model_name)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/351 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [7]:
dataset = df.loc[df['title'] == 'Location data of worker bumblebees across an agricultural landscape in Buckinghamshire, UK']
context = dataset.iloc[0]['description']

In [8]:
import pandas as pd
questions = ['How was DNA sampling performed?', 'Who funded the project?', 'Where was the data gathered?']
results = pd.DataFrame(columns=['Question'] + model_names)
for q in questions:
    result = {'Question': q}
    for model_name in model_names:
        ans = qa_pipes[model_name](q, context)
        result[model_name] = f"{ans['answer']} ({ans['score']:.2f})"
    results = results._append(result, ignore_index=True)
print(results.transpose().to_markdown())

|                                                  | 0                                | 1                                     | 2                                            |
|:-------------------------------------------------|:---------------------------------|:--------------------------------------|:---------------------------------------------|
| Question                                         | How was DNA sampling performed?  | Who funded the project?               | Where was the data gathered?                 |
| deepset/roberta-base-squad2                      | non-lethally (0.13)              | Centre for Ecology & Hydrology (0.27) | Hillesden Estate, Buckinghamshire, UK (0.16) |
| distilbert/distilbert-base-cased-distilled-squad | handheld GPS unit (0.26)         | Insect Pollinators Initiative (0.21)  | Hillesden Estate, Buckinghamshire, UK (0.54) |
| deepset/minilm-uncased-squad2                    | non-lethally (0.81)              | Insect Pollinators Initiative (0.62) 

In [9]:
from transformers import pipeline
import pandas as pd
gen_model_names = ['google-t5/t5-base', 'allenai/unifiedqa-v2-t5-large-1363200']
gen_models = {}
for model_name in gen_model_names:
    gen_models[model_name] = pipeline('text2text-generation', model=model_name)



In [10]:
results = pd.DataFrame(columns=['Question'] + gen_model_names)
for q in questions:
    result = {'Question': q}
    for model_name in gen_model_names:
        ans = gen_models[model_name](f'question: {q} context: {context}')
        result[model_name] = f"{ans[0]['generated_text']}"
    results = results._append(result, ignore_index=True)
print(results.transpose().to_markdown())

/home/mpc/github/discoverability/.venv/lib/python3.12/site-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/mpc/github/discoverability/.venv/lib/python3.12/site-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/mpc/github/discoverability/.venv/lib/python3.12/site-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


|                                       | 0                               | 1                              | 2                                     |
|:--------------------------------------|:--------------------------------|:-------------------------------|:--------------------------------------|
| Question                              | How was DNA sampling performed? | Who funded the project?        | Where was the data gathered?          |
| google-t5/t5-base                     | non-lethally                    | Insect Pollinators Initiative  | Hillesden Estate, Buckinghamshire, UK |
| allenai/unifiedqa-v2-t5-large-1363200 | non-lethally                    | insect pollinators initiative. | the Hillesden Estate                  |
